# Does Testing Save Time?

$T = r_{\rm code} l + r_{\rm test} l + t_{\rm DB}$,
where
* $T \equiv$ total time spent coding
* $l \equiv$ number of lines of code
* $r_{\rm code} \equiv$ time spent writing the code itself per line of code
* $r_{\rm test} \equiv$ time spent writing the testing code per line of code
* $t_{\rm DB} \equiv$ time spent debugging

### Time spent Debugging

Let's breakdown that last term a little more...

$t_{\rm DB} = n_l t_l + n_m t_m$, where
* $n_l \equiv$ number of bugs that are easily locatable
* $t_l \equiv$ time spent fixing an easily locatable bug
* $n_m \equiv$ number of bugs that are in a mysterious, hard-to-find location
* $t_m \equiv$ time spent fixing a hard-to-find bug

With a bit of re-ordering this becomes

$t_{\rm DB} = n_{\rm caught} t_l ( 1 + f_m \frac{( t_m - t_l )}{t_l} )$, where
* $n_{\rm caught} \equiv$ number of bugs caught
* $f_m \equiv$ fraction of bugs that are in a mystery location.

Finally we can break down $n_{\rm caught}$ into

$n_{\rm caught} \equiv f_{c} n = f_{c} e l$, where
* $f_{c} \equiv$ the fraction of bugs that are caught
* $n \equiv$ total number of bugs in the code
* $e \equiv$ number of bugs per line of code 

Therefore

$t_{\rm DB} = l f_c e t_l ( 1 + f_m \frac{( t_m - t_l )}{t_l} )$

## Final parametrized form to explore

Putting everything together, we get

$T/l = r_{\rm code} + r_{\rm test} + f_c e t_l ( 1 + f_m \frac{( t_m - t_l )}{t_l} )$

When we do/don't do testing we can expect that the only values that change are $r_{\rm test}$, $f_c$, and $f_m$.
Therefore we can explore the ratio

$T_{\rm testing} / T_{\rm no~testing} = (r_{\rm code} + r_{\rm test} + f_{c,t} e t_l ( 1 + f_{m,t} \frac{( t_m - t_l )}{t_l} )) / (r_{\rm code} + f_{c,n} e t_l ( 1 + f_{m,n} \frac{( t_m - t_l )}{t_l} ))$

## Import Estimates

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('~/Downloads/Coding Habits Survey (Responses Cleaned) - Form Responses 1.csv', header=0,)

In [ ]:
df = df.drop( 0 )

## Let's actually explore the function

In [ ]:
def time_coding_per_line( r_code, r_test, e, t_l, t_m, f_c, f_m,  ):
        
    return r_code + r_test + f_c * e * t_l * ( 1. + f_m * ( t_m - t_l ) / t_l )

In [ ]:
def test_vs_no_test( r_code, r_test, e, t_l, t_m, f_c_test, f_m_test, f_c_notest, f_m_notest ):
    
    results = []
    for i, (f_c, f_m) in enumerate( zip( [ f_c_test, f_c_notest ], [ f_m_test, f_m_notest ] ) ):
        
        # During the new-test case there should be no time spent testing, ofc
        if i == 1:
            r_test = 0.
                
        results.append( time_coding_per_line( r_code, r_test, e, t_l, t_m, f_c, f_m,  ) )
        
    return results

In [ ]:
estimated_parameters = {
    'r_code': df['r_code'].values.astype( float ),
    'r_test': df['r_code'].values.astype( float ), # We'll assume it takes just as long to test as to code, a conservative assumption
    'e': df['e'].values.astype( float ),
    't_l': df['t_l'].values.astype( float ),
    't_m': df['t_m'].values.astype( float ),
    'f_c_notest': 1. - df['1 - f_c_notest'].values.astype( float ),
    'f_m_notest': 1. - df['1 - f_m_notest'].values.astype( float ),
    'f_c_test': 1. - df['1 - f_c_test'].values.astype( float ),
    'f_m_test': 1. - df['1 - f_m_test'].values.astype( float ),
}

In [ ]:
estimated_parameters

In [ ]:
# Don't allow f_c_test to fall below f_c_no_test
invalid = estimated_parameters['f_c_test'] <= estimated_parameters['f_c_notest']
# Assume we catch half the remaining bugs
estimated_parameters['f_c_test'][invalid] = (
    estimated_parameters['f_c_notest'][invalid] +
    ( 1. - estimated_parameters['f_c_notest'][invalid] ) * 0.5
)
print( 'Fixed {} invalid f_c_test values'.format( invalid.sum() ) )

# Don't allow f_m_notest to fall below f_m_test
invalid = estimated_parameters['f_m_test'] >= estimated_parameters['f_m_notest']
# Assume half the remaining bugs are now easy to find
estimated_parameters['f_m_test'][invalid] = estimated_parameters['f_m_notest'][invalid] * 0.5
print( 'Fixed {} invalid f_m_test values'.format( invalid.sum() ) )


In [ ]:
# Reformat
import verdict
responses = {}
for key, item in estimated_parameters.items():
    responses[key] = {}
    for i, v in enumerate( item ):
        responses[key][i] = v
responses = verdict.Dict( responses ).transpose()

In [ ]:
# Get results
for i, parameters in responses.items():
    result = test_vs_no_test( **parameters )
    print( '{:>7.2g} min per line w/ testing, {:>7.2g} min per line w/o, {:>7.2g} ratio, {:>7.2g} df_m, {:>7.2g} df_c'.format(
            result[0], 
            result[1], 
            result[0]/result[1],
            parameters['f_m_test'] - parameters['f_m_notest'],
            parameters['f_c_test'] - parameters['f_c_notest'],
        )
    )
    responses[i]['T/l test'], responses[i]['T/l notest'] = result